# Brightness Adjustment - CPU Kernel Operation for Images
This script demonstrates a basic concept of image processing - adjust the brightness of each pixel in an image.

Brightness adjustment is one of the simplest image processing operations - we add a constant value to every pixel.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time

## Preparing Data

In [ ]:
# create a small test image with gradient pattern
rows, cols = 6, 8
test_image = np.zeros((rows, cols), dtype=np.float32)

# fill with gradient from dark (top-left) to bright (bottom-right)
for r in range(rows):
    for c in range(cols):
        test_image[r, c] = (r * cols + c) * 4  

print(f"Test image ({rows}×{cols}):")
print(test_image)

fig, ax = plt.subplots()
im = ax.matshow(test_image, cmap='gray', vmin=0, vmax=255)
plt.colorbar(im, ax=ax, fraction=0.046)

## Sequential Implementation

In [ ]:
def adjust_brightness_sequential(image, brightness_offset):
    """
    Sequential implementation processes one pixel at a time.
    This demonstrates how a CPU would execute the kernel.
    This includes clamping to valid range [0, 255] for 8-bit images.
    """
    num_rows, num_cols = image.shape
    result = np.zeros((num_rows, num_cols), dtype=np.float32)
    
    # process each pixel individually
    for row in range(num_rows):
        for col in range(num_cols):
            # apply the kernel function to pixel at (row, col)
            new_value = image[row, col] + brightness_offset
            
            # clamp to valid range
            if new_value < 0:
                new_value = 0
            elif new_value > 255:
                new_value = 255
                
            result[row, col] = new_value
   
    return result

In [ ]:
brightness_offset = 50.0
result = adjust_brightness_sequential(test_image, brightness_offset)

# visualise input and output
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

im1 = ax1.matshow(test_image, cmap='gray', vmin=0, vmax=255)
ax1.set_title(f"Original Image")
ax1.set_xlabel("Column")
ax1.set_ylabel("Row")
plt.colorbar(im1, ax=ax1, fraction=0.046)

im2 = ax2.matshow(result, cmap='gray', vmin=0, vmax=255)
ax2.set_title(f"Brightness +{int(brightness_offset)}")
ax2.set_xlabel("Column")
ax2.set_ylabel("Row")
plt.colorbar(im2, ax=ax2, fraction=0.046)

plt.tight_layout()
plt.show()

print(f"\nOriginal image:")
print(test_image)
print(f"\nAfter brightness adjustment:")
print(result)

## Simulating Parallel Processing

In [ ]:
def simulate_parallel_processing_2d(image, brightness_offset, block_shape=(2, 2)):
    """
    Simulates how parallel processing would work on a GPU.
    Each thread block processes a portion of the image independently.
    """
    num_rows, num_cols = image.shape
    block_rows, block_cols = block_shape
    
    # calculate grid dimensions
    grid_rows = (num_rows + block_rows - 1) // block_rows
    grid_cols = (num_cols + block_cols - 1) // block_cols
    
    result = np.zeros_like(image)
    block_assignments = []
    
    # simulate each thread block processing its region
    for block_y in range(grid_rows):
        for block_x in range(grid_cols):
            # calculate which pixels this block processes
            start_row = block_y * block_rows
            end_row = min(start_row + block_rows, num_rows)
            start_col = block_x * block_cols
            end_col = min(start_col + block_cols, num_cols)
            
            # record assignment for visualisation
            block_assignments.append(((block_y, block_x), 
                                     (start_row, end_row), 
                                     (start_col, end_col)))
            
            # process this block's pixels
            for row in range(start_row, end_row):
                for col in range(start_col, end_col):
                    new_value = image[row, col] + brightness_offset
                    # clamp
                    if new_value < 0:
                        new_value = 0
                    elif new_value > 255:
                        new_value = 255
                    result[row, col] = new_value
    
    return result, block_assignments

In [ ]:
result_parallel, assignments = simulate_parallel_processing_2d(
    test_image, brightness_offset, block_shape=(3, 4)
)

print(f"Using {len(assignments)} thread blocks:")
for (block_y, block_x), (sr, er), (sc, ec) in assignments:
    print(f"  Block ({block_y},{block_x}): processes pixels "
          f"rows {sr}-{er-1}, cols {sc}-{ec-1}")

# verify results match
print(f"\nResults match sequential: {np.allclose(result, result_parallel)}")

## Visualise Parallel Processing

In [ ]:
def visualise_processing_2d(image_shape=(8, 12), block_shape=(4, 4)):
    """Create a visual representation of sequential vs parallel processing for 2D images."""
    rows, cols = image_shape
    block_rows, block_cols = block_shape
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # sequential processing visualisation
    ax1.set_title("Sequential Processing (CPU)", fontsize=14)
    ax1.set_xlim(-0.5, cols - 0.5)
    ax1.set_ylim(-0.5, rows - 0.5)
    ax1.set_xlabel("Column")
    ax1.set_ylabel("Row")

    # match image coordinates
    ax1.invert_yaxis()  
    
    # create gradient showing processing order
    sequential_order = np.zeros((rows, cols))
    for r in range(rows):
        for c in range(cols):
            sequential_order[r, c] = r * cols + c
    
    im1 = ax1.matshow(sequential_order, cmap='viridis', alpha=0.7)
    
    # add zig-zag arrows showing sequential row-major flow
    arrow_props = dict(arrowstyle='->', color='red', lw=2)
    
    for r in range(rows):
        # each row processes left to right
        if cols > 1:  
            # only add arrow if there are multiple columns
            ax1.annotate('', xy=(cols-1, r), xytext=(0, r),
                         arrowprops=arrow_props)
        
        # connect last pixel of current row to first pixel of next row
        if r < rows - 1:
            ax1.annotate('', xy=(0, r+1), xytext=(cols-1, r),
                         arrowprops=dict(arrowstyle='->', color='red', lw=1.5))
    
    ax1.text(cols/2, rows + 0.3, 'Processes pixels in row-major order',
             ha='center', color='red', fontsize=12, weight='bold',
             bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))

    
    # parallel processing visualisation
    ax2.set_title("Parallel Processing (GPU)", fontsize=14)
    ax2.set_xlim(-0.5, cols - 0.5)
    ax2.set_ylim(-0.5, rows - 0.5)
    ax2.set_xlabel("Column")
    ax2.set_ylabel("Row")
    ax2.invert_yaxis()
    
    # calculate grid
    grid_rows = (rows + block_rows - 1) // block_rows
    grid_cols = (cols + block_cols - 1) // block_cols
    
    # colour map for blocks
    colours = plt.cm.Set3(np.linspace(0, 1, grid_rows * grid_cols))
    
    # show blocks
    block_id = 0
    for br in range(grid_rows):
        for bc in range(grid_cols):
            start_row = br * block_rows
            end_row = min(start_row + block_rows, rows)
            start_col = bc * block_cols
            end_col = min(start_col + block_cols, cols)
            
            # create block patch
            block_array = np.ones((end_row - start_row, end_col - start_col))
            block_extent = [start_col - 0.5, end_col - 0.5, 
                           end_row - 0.5, start_row - 0.5]
            
            ax2.imshow(block_array, extent=block_extent,
                      cmap='gray', vmin=0, vmax=2,
                      alpha=0)
            
            # draw block boundary
            rect = plt.Rectangle((start_col - 0.5, start_row - 0.5),
                               end_col - start_col, end_row - start_row,
                               fill=False, edgecolor=colours[block_id],
                               linewidth=3)
            ax2.add_patch(rect)
            
            # label the block
            block_center_x = (start_col + end_col - 1) / 2
            block_center_y = (start_row + end_row - 1) / 2
            ax2.text(block_center_x, block_center_y,
                    f"Block\n({br},{bc})",
                    ha='center', va='center', fontsize=10,
                    weight='bold', color=colours[block_id])
            
            block_id += 1
    
    ax2.text(cols/2, -1.5, 
            'All blocks process simultaneously',
            ha='center', color='green', fontsize=12, weight='bold',
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
    
    # add grid lines
    for ax in [ax1, ax2]:
        ax.set_xticks(range(cols))
        ax.set_yticks(range(rows))
        ax.grid(True, alpha=0.3, linewidth=0.5)
    
    plt.tight_layout()
    plt.show()

In [ ]:
visualise_processing_2d(image_shape=(8, 12), block_shape=(4, 4))

## Performance Comparison

In [ ]:
def benchmark_implementations(sizes):
    """Compare performance of sequential and vectorised implementations."""
    results = {
        "sequential": [],
        "vectorised": [],
        "sizes": sizes
    }
    brightness_offset = 75.0
    
    print("\nPerformance Comparison")
    print("=" * 60)
    print(f"{'Image Size':>15} {'Sequential':>12} {'Vectorised':>12} {'Speed-up':>12}")
    print("-" * 60)

    rng = np.random.default_rng()
    for size in sizes:
        # create test image
        image = rng.integers(0, 200, size=(size, size), dtype=np.uint8).astype(np.float32)
        
        # sequential implementation
        start = time.perf_counter()
        result_seq = adjust_brightness_sequential(image, brightness_offset)
        time_seq = time.perf_counter() - start
        results["sequential"].append(time_seq)
        
        # vectorised implementation with clamping
        start = time.perf_counter()
        result_vec = image + brightness_offset
        result_vec = np.clip(result_vec, 0, 255)
        time_vec = time.perf_counter() - start
        results["vectorised"].append(time_vec)
        
        # verify results match
        assert np.allclose(result_seq, result_vec)
        
        print(f"{size:>4}   x   {size:<4} {time_seq:>12.6f} {time_vec:>12.6f} {time_seq/time_vec:>12.2f}x")
    
    return results

In [ ]:
sizes = [32, 64, 128, 256, 512]
results = benchmark_implementations(sizes)

## Demonstrating Clamping Behaviour

In [ ]:
# create test image with extreme values to show clamping
extreme_image = np.array([
    [5,   10,  20,  30],    # very dark pixels
    [50,  100, 150, 200],   # medium pixels
    [210, 220, 230, 240],   # bright pixels  
    [245, 250, 253, 255]    # very bright pixels
], dtype=np.float32)

# test with large positive offset
bright_offset = 100
brightened = adjust_brightness_sequential(extreme_image, bright_offset)

# test with negative offset
dark_offset = -100
darkened = adjust_brightness_sequential(extreme_image, dark_offset)

print("Demonstrating clamping behaviour:")
print("=" * 50)
print(f"\nOriginal image:")
print(extreme_image)

print(f"\nAfter brightness +{bright_offset}:")
print(brightened)
print("Note: Values clamped at 255")

print(f"\nAfter brightness {dark_offset}:")
print(darkened)
print("Note: Values clamped at 0")

# visualise clamping
fig, ax = plt.subplots(1, 3, figsize=(12, 4))

im1 = ax[0].matshow(extreme_image, cmap='gray', vmin=0, vmax=255)
ax[0].set_title("Original")
plt.colorbar(im1, ax=ax[0], fraction=0.046)

im2 = ax[1].matshow(brightened, cmap='gray', vmin=0, vmax=255)
ax[1].set_title(f"Brightness +{bright_offset}")
plt.colorbar(im2, ax=ax[1], fraction=0.046)

im3 = ax[2].matshow(darkened, cmap='gray', vmin=0, vmax=255)
ax[2].set_title(f"Brightness {dark_offset}")
plt.colorbar(im3, ax=ax[2], fraction=0.046)

for a in ax:
    a.set_xlabel("Column")
    a.set_ylabel("Row")

plt.tight_layout()
plt.show()